<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d5ba33b4cb24e63618968a283965ded54a75c41fc9682369b94d9ad3e8648100
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 13:48:20
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 28.43 K (0.2%)
Current PnL: -26.14 L (-16.88%)
CY Booked + Current PnL: -11.85 L (-7.65%)
-------------------
Total profit:  1.45 L
Total loss:  -27.59 L
-------------------
Total Booked + Current PnL: 15.13 L (11.84%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.14%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.77 L (64.42%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.16,-16.60,20.01,0.09,16818.0,-16727.0,84050.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.60,10.42,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.16,-12.21,13.93,0.02,21348.0,-21307.0,153252.0,133.82,53.0,M-SC,15.34,234.0,-1.00,1.09,33.30,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.91,-0.18,22.27,22.05,21780.0,-180.0,97800.0,-13.73,60.0,H-SC,7.20,164.0,-0.01,0.70,42.16,XR,ATH,FINANCE
82,VOLTAS,1530.00,0.49,8.26,10.56,19.69,21921.0,15843.0,207585.0,-2.32,56.0,X-MC,2.56,78.0,0.72,1.48,15.62,XY25,NTT,AC
43,ITC,452.00,0.72,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,48.0,X-LC,1.90,5.0,-0.10,1.69,3.94,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,12.62,-37.96,118.04,35.27,83153.0,-43104.0,70445.0,-51.71,54.0,L-SC,10.81,270.0,-0.52,0.50,72.42,XR,NTT,HOTELS
57,RAJOOENG,143.10,7.61,-33.37,109.52,39.61,74802.0,-34200.0,68300.0,-54.88,39.0,H-SC,28.01,135.0,-0.46,0.49,7.61,AR,ATH,MISC
44,JCHAC,2282.00,3.19,-35.46,54.92,-0.01,45302.0,-45317.0,82488.0,16769.57,32.0,M-SC,6.64,233.0,-1.00,0.59,5.58,OX40N,NTT,AC
22,DIXON,18931.72,2.68,-8.52,37.82,26.07,67539.0,-16640.0,178581.0,-55.55,42.0,X-MC,8.31,56.0,-0.25,1.27,11.22,X40N,ATH,IT
79,VAIBHAVGBL,521.00,2.52,-25.28,115.16,60.77,157277.0,-46202.0,136573.0,57.58,51.0,H-SC,4.31,159.0,-0.29,0.97,22.99,XR,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.53,-17.88,121.22,81.67,93389.0,-16769.0,77041.0,6922.22,41.0,L-SC,12.95,271.0,-0.18,0.55,49.19,XR,NTT,CERAMICS
33,HINDUNILVR,2922.00,-1.95,-12.06,29.26,13.67,66145.0,-31003.0,226060.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.61,6.72,XY25,NTT,FMCG
14,CAMPUS,393.00,-1.61,-12.13,51.68,33.28,76191.0,-20347.0,147428.0,-29.23,36.0,M-SC,2.63,222.0,-0.27,1.05,14.27,XY24,NTT,FOOTWEAR
16,CERA,9475.00,-1.16,-32.04,82.30,23.89,98383.0,-56361.0,119542.0,-34.37,28.0,H-SC,10.29,157.0,-0.57,0.85,3.24,OX40N,NTT,CERAMICS
67,SONACOMS,806.63,-1.09,-15.73,65.58,39.54,55908.0,-15908.0,85251.0,-34.10,49.0,M-SC,4.64,220.0,-0.28,0.61,20.21,AR,ATH,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,0.91,-0.18,22.27,22.05,21780.0,-180.0,97800.0,-13.73,60.0,H-SC,7.2,164.0,-0.01,0.7,42.16,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.07,-5.26,119.44,107.91,167079.0,-7759.0,139885.0,-22.98,46.0,M-SC,12.65,216.0,-0.05,1.00,4.30,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.16,-12.21,13.93,0.02,21348.0,-21307.0,153252.0,133.82,53.0,M-SC,15.34,234.0,-1.00,1.09,33.30,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.16,-16.60,20.01,0.09,16818.0,-16727.0,84050.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.60,10.42,OX40N,NTT,DURABLES
66,SIS,528.00,1.88,-23.99,59.54,21.26,50247.0,-26640.0,84392.0,1991.19,51.0,H-SC,4.84,166.0,-0.53,0.60,14.02,OX40N,NTT,MISC
47,KANSAINER,340.00,1.51,-24.28,49.86,13.47,101810.0,-65475.0,204192.0,-68.90,38.0,H-SC,4.73,158.0,-0.64,1.46,3.83,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.04,6.93,61.1,72.27,110705.0,11738.0,181186.0,-8.35,68.0,M-LC,2.21,99.0,0.11,1.29,13.53,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.04,6.93,61.10,72.27,110705.0,11738.0,181186.0,-8.35,68.0,M-LC,2.21,99.0,0.11,1.29,13.53,XR,NTT,IT
50,MASFIN,398.61,0.91,-0.18,22.27,22.05,21780.0,-180.0,97800.0,-13.73,60.0,H-SC,7.20,164.0,-0.01,0.70,42.16,XR,ATH,FINANCE
38,INDIAMART,4810.62,1.94,-2.22,111.41,106.72,134363.0,-2734.0,120602.0,-51.58,41.0,H-SC,2.65,139.0,-0.02,0.86,20.64,AR,ATH,MISC
25,FINCABLES,1641.55,1.07,-5.26,119.44,107.91,167079.0,-7759.0,139885.0,-22.98,46.0,M-SC,12.65,216.0,-0.05,1.00,4.30,OX40N,ATH,CABLES
37,IEX,219.00,-0.13,-2.95,53.14,48.63,103126.0,-5889.0,194065.0,-35.53,52.0,H-SC,14.03,136.0,-0.06,1.38,8.27,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.08,-18.97,91.86,55.47,181098.0,-46140.0,197146.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,1.01,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-1.95,-12.06,29.26,13.67,66145.0,-31003.0,226060.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.61,6.72,XY25,NTT,FMCG
53,PAGEIND,51605.07,-0.56,-8.83,40.52,28.11,74405.0,-17785.0,183625.0,-33.88,30.0,X-MC,12.66,55.0,-0.24,1.31,0.08,X40,ATH,APPARELS
75,TMPV,600.00,-0.27,-28.98,73.16,22.98,117371.0,-65468.0,160430.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.90,XY24,NTT,AUTO
23,DMART,5391.45,-0.42,-8.01,40.89,29.60,93887.0,-19992.0,229608.0,-21.58,32.0,X-LC,5.48,19.0,-0.21,1.64,12.42,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.15,-0.90,25.61,24.48,63917.0,-2277.0,249579.0,-81.09,45.0,X-SC,0.69,86.0,-0.04,1.78,22.03,X40N,NTT,MISC
43,ITC,452.0,0.72,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,48.0,X-LC,1.90,5.0,-0.10,1.69,3.94,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.26,6.87,22.32,30.73,46994.0,13542.0,210547.0,-18.17,63.0,X-MC,2.12,75.0,0.29,1.50,20.52,X40,ATH,INSURANCE
3,ACC,3906.0,0.40,-25.25,119.59,64.15,212727.0,-60071.0,177880.0,-56.31,36.0,X-SC,2.42,82.0,-0.28,1.27,0.40,XY24,BTT,CEMENT
33,HINDUNILVR,2922.0,-1.95,-12.06,29.26,13.67,66145.0,-31003.0,226060.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.61,6.72,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.56,-8.83,40.52,28.11,74405.0,-17785.0,183625.0,-33.88,30.0,X-MC,12.66,55.0,-0.24,1.31,0.08,X40,ATH,APPARELS
3,ACC,3906.00,0.40,-25.25,119.59,64.15,212727.0,-60071.0,177880.0,-56.31,36.0,X-SC,2.42,82.0,-0.28,1.27,0.40,XY24,BTT,CEMENT
75,TMPV,600.00,-0.27,-28.98,73.16,22.98,117371.0,-65468.0,160430.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.90,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-1.08,-18.97,91.86,55.47,181098.0,-46140.0,197146.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,1.01,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.09,-38.11,118.47,35.20,94338.0,-49040.0,79630.0,4.40,34.0,X-SC,15.99,92.0,-0.52,0.57,1.15,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.65,-28.96,101.06,42.84,46673.0,-18823.0,46183.0,-53.72,45.0,X-SC,35.90,83.0,-0.40,0.33,4.48,XY24,NTT,MISC
59,RELAXO,1176.00,-0.11,-47.60,190.66,52.31,145026.0,-69095.0,76065.0,-44.25,42.0,X-SC,6.71,91.0,-0.48,0.54,1.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.09,-38.11,118.47,35.20,94338.0,-49040.0,79630.0,4.40,34.0,X-SC,15.99,92.0,-0.52,0.57,1.15,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.62,-4.37,29.75,24.08,36117.0,-5548.0,121402.0,-12.07,33.0,X-SC,5.23,89.0,-0.15,0.87,14.39,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.52,-16.33,71.29,43.32,97154.0,-26592.0,136280.0,-28.06,38.0,X-SC,4.62,90.0,-0.27,0.97,4.64,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.08,-11.38,36.20,20.71,121343.0,-43035.0,335202.0,-23.62,64.0,X-LC,4.66,1.0,-0.35,2.39,12.00,X40,ATH,IT
41,INFY,1972.00,0.73,9.59,22.49,34.24,77120.0,30021.0,342909.0,-14.57,65.0,X-LC,3.60,2.0,0.39,2.45,18.67,X40,NTT,IT
75,TMPV,600.00,-0.27,-28.98,73.16,22.98,117371.0,-65468.0,160430.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.90,XY24,NTT,AUTO
81,VBL,671.64,-0.88,-3.90,41.17,35.67,124962.0,-12314.0,303528.0,-15.30,55.0,X-LC,4.49,4.0,-0.10,2.16,9.42,X40N,ATH,FMCG
43,ITC,452.00,0.72,-1.23,12.16,10.78,28864.0,-2951.0,237367.0,-41.15,48.0,X-LC,1.90,5.0,-0.10,1.69,3.94,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.63,-37.13,111.39,32.89,54064.0,-28670.0,48536.0,-699.18,57.0,L-MC,5.98,259.0,-0.53,0.35,33.66,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.53,-17.88,121.22,81.67,93389.0,-16769.0,77041.0,6922.22,41.0,L-SC,12.95,271.0,-0.18,0.55,49.19,XR,NTT,CERAMICS
50,MASFIN,398.61,0.91,-0.18,22.27,22.05,21780.0,-180.0,97800.0,-13.73,60.0,H-SC,7.20,164.0,-0.01,0.70,42.16,XR,ATH,FINANCE
13,BSOFT,831.70,0.81,-21.46,91.99,50.79,101618.0,-30187.0,110466.0,0.89,68.0,H-SC,6.08,151.0,-0.30,0.79,28.62,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.16,-12.21,13.93,0.02,21348.0,-21307.0,153252.0,133.82,53.0,M-SC,15.34,234.0,-1.00,1.09,33.30,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.81,-21.46,91.99,50.79,101618.0,-30187.0,110466.0,0.89,68.0,H-SC,6.08,151.0,-0.30,0.79,28.62,XR,ATH,IT
84,WIPRO,420.00,0.04,6.93,61.10,72.27,110705.0,11738.0,181186.0,-8.35,68.0,M-LC,2.21,99.0,0.11,1.29,13.53,XR,NTT,IT
41,INFY,1972.00,0.73,9.59,22.49,34.24,77120.0,30021.0,342909.0,-14.57,65.0,X-LC,3.60,2.0,0.39,2.45,18.67,X40,NTT,IT
32,HCLTECH,1922.01,0.74,9.39,14.03,24.73,37129.0,22715.0,264639.0,14.03,67.0,X-LC,6.73,7.0,0.61,1.89,26.06,X40,ATH,IT
1,ABB,7934.00,0.12,-3.04,50.14,45.57,127183.0,-7963.0,253656.0,-37.68,60.0,H-MC,4.58,120.0,-0.06,1.81,7.93,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.32
1,25,44.87
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.92
LC    24.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.80
X40      22.95
X40N     13.01
XR        9.83
AR        9.06
XY25      9.06
OX40N     7.53
SR        0.98
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.41
H-SC    23.91
X-LC    20.62
M-SC    11.66
X-SC     8.05
H-MC     4.82
L-SC     1.42
M-MC     1.34
M-LC     1.29
H-LC     1.12
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-7.54,42.12
IT,13.04,-14.98,72.80
FINANCE,9.55,-19.15,70.96
MISC,7.33,-26.34,78.01
ELECTRICAL,6.02,-11.15,51.87
PAINTS,5.85,-12.50,29.21
INSURANCE,4.77,-1.22,35.86
PHARMA,4.11,-1.79,33.83
AUTO,2.82,-32.57,78.52


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3164355.0,21
AR,1302368.0,10
XR,1283887.0,13
X40,1003602.0,14
X40N,860037.0,9
OX40N,736290.0,10
XY25,370381.0,6
SR,281794.0,2
MH,74604.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3627246.0,25
M-SC,1423100.0,15
X-MC,1402909.0,16
X-LC,833982.0,11
X-SC,790491.0,8
H-MC,401687.0,3
L-SC,262373.0,3
M-LC,110705.0,1
H-LC,76053.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1265302.0      6
           AR         892768.0      5
M-SC       XY24       816758.0      6
H-SC       XR         769902.0      7
X-MC       X40        490406.0      7
           XY24       399764.0      3
H-SC       OX40N      342876.0      4
X-MC       X40N       335091.0      4
X-LC       X40        324319.0      5
M-SC       OX40N      307583.0      5
X-SC       X40N       306097.0      3
           XY24       295517.0      3
H-SC       SR         281794.0      2
X-LC       X40N       218849.0      2
H-MC       AR         207462.0      2
           XY24       194225.0      1
X-LC       XY24       192789.0      2
X-SC       X40        188877.0      2
X-MC       XY25       177648.0      2
L-SC       XR         176542.0      2
M-SC       XR         172674.0      2
           AR         126085.0      2
M-LC       XR         110705.0      1
X-LC       XY25        98025.0      2
L-SC       OX40N       85831.0      1
H-LC       AR          76053.0      1
H-SC       MH          74604.0      1
M-MC       XY25        57445.0      1
L-MC       XR          54064.0      1
L-LC       XY25        37263.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
